In [4]:
import os
import random
import shutil, os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt   
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import pylab as pl
from matplotlib.pyplot import figure
from collections import Counter

from os import listdir
from os.path import isfile, join

In [5]:
truth_file = pd.read_csv('/Users/sleung2/Documents/MIDS Program/Capstone_local/snapshot_wisconsin/all/yolo_splits3/test/test_labels.csv')
truth_file = truth_file.rename(columns = {'TRIGGER_ID': 'event_id',
                                               'CLASS_SPECIES': 'true_class',
                                               'Total': 'true_count',
                                               'CLASS_SPECIES_RESTATED': 'true_class_name'})

truth_file_no_other = truth_file[truth_file['true_class_name'] != 'other']

In [6]:
labels = pd.DataFrame(['foxgray_foxred',
              'cottontail_snowshoehare',
              'raccoon',
              'opossum',
              'turkey',
              'bear',
              'elk',
              'deer',
              'coyote',
              'wolf']).sort_values(0)
labels = labels.rename(columns = {0: 'species'})
labels.insert(0, 'class', range(0, len(labels)))

# Phase 1- Blank

## Yolo

In [26]:
#Read lines from txt results file

toppath = 'phase 1-blank/'
txt_file = 'phase1_yolo.txt'

with open(toppath + txt_file, 'r') as f:
    blank_yolo_lines = f.readlines()

In [130]:
yolo_image_prediction_list = []
yolo_blank_prediction_list = []


for line in blank_yolo_lines:
    
    image_name = line.split('Filename: ')[1].split('; ')[0]
    prediction = line.split('; ')[1].split('; Bbox[list]')[0]
    yolo_image_prediction_list.append(image_name)
    yolo_blank_prediction_list.append(prediction)

In [133]:
yolo_blank_prediction_list

['',
 'Class:0.0,Conf:0.5518747568130493;',
 '',
 'Class:0.0,Conf:0.8673955798149109;',
 'Class:0.0,Conf:0.7018851041793823;',
 'Class:0.0,Conf:0.2550954222679138;',
 'Class:0.0,Conf:0.8825900554656982;',
 'Class:0.0,Conf:0.2834002673625946;',
 'Class:0.0,Conf:0.7554515600204468;',
 'Class:0.0,Conf:0.8432669043540955;',
 'Class:0.0,Conf:0.8680070638656616;',
 '',
 '',
 '',
 'Class:0.0,Conf:0.7652684450149536;',
 'Class:0.0,Conf:0.8384182453155518;',
 '',
 'Class:0.0,Conf:0.50357586145401;Class:0.0,Conf:0.5593075156211853;Class:0.0,Conf:0.6616421937942505;',
 '',
 'Class:0.0,Conf:0.42296430468559265;',
 '',
 'Class:0.0,Conf:0.3392694592475891;',
 'Class:0.0,Conf:0.8630671501159668;',
 'Class:0.0,Conf:0.7669381499290466;',
 'Class:0.0,Conf:0.752083420753479;Class:0.0,Conf:0.7951993346214294;',
 '',
 'Class:0.0,Conf:0.8413991332054138;',
 'Class:0.0,Conf:0.5667402148246765;',
 'Class:0.0,Conf:0.5962253212928772;',
 'Class:0.0,Conf:0.8641231060028076;',
 'Class:0.0,Conf:0.8275712728500366;

In [137]:
yolo_stage_1_prediction_df = pd.DataFrame({'image_name': yolo_image_prediction_list, 'prediction': yolo_blank_prediction_list})



In [140]:
yolo_stage_1_prediction_df.sort_values(by = 'image_name')

,image_name,prediction,event_id
1743,SSWI000000002741773A.jpg,,SSWI000000002741773
3471,SSWI000000002741773B.jpg,,SSWI000000002741773
3270,SSWI000000002741773C.jpg,,SSWI000000002741773
578,SSWI000000004032002A.jpg,"Class:0.0,Conf:0.8916367292404175;",SSWI000000004032002
3220,SSWI000000004032002B.jpg,"Class:0.0,Conf:0.8409765362739563;",SSWI000000004032002
...,...,...,...
2245,SSWI000000023514111B.jpg,"Class:0.0,Conf:0.694103479385376;",SSWI000000023514111
2956,SSWI000000023514111C.jpg,"Class:0.0,Conf:0.8248010277748108;",SSWI000000023514111
1527,SSWI000000023514155A.jpg,"Class:0.0,Conf:0.3264309763908386;Class:0.0,Co...",SSWI000000023514155
1003,SSWI000000023514155B.jpg,"Class:0.0,Conf:0.7902511358261108;",SSWI000000023514155


In [139]:
yolo_stage_1_prediction_df['event_id'] = yolo_stage_1_prediction_df['image_name'].apply(lambda x: x[:-5] if 'SSWI' in x else x.split('_')[0])

In [141]:
yolo_stage_1_prediction_df['class_name'] = yolo_stage_1_prediction_df['prediction'].apply(lambda x: 'empty' if x == '' else 'animal')
yolo_stage_1_prediction_df['class'] = yolo_stage_1_prediction_df['class_name'].apply(lambda x: 0 if x == 'animal' else 1)



In [143]:
yolo_stage_1_prediction_grouped = pd.pivot_table(yolo_stage_1_prediction_df, index = 'event_id',
              columns = 'class_name',
              values = 'class',
              aggfunc=len, fill_value=0).reset_index()



## Effnet

In [97]:
#Read lines from json results file
    
effnet_stage_1 = pd.read_json(toppath + 'phase1_efficientnetb0_classifications.json')
effnet_stage_1 = effnet_stage_1['phase1_classification_results'].apply(pd.Series)


effnet_stage_1['event_id'] = effnet_stage_1['id'].apply(lambda x: x[:-5] if 'SSWI' in x else x.split('_')[0])


In [98]:
effnet_stage_1_pivot = pd.pivot_table(effnet_stage_1, index = 'event_id',
              columns = 'class_name',
              values = 'class',
              aggfunc=len, fill_value=0) #len(x.unique())).fillna(0)



In [99]:
effnet_stage_1_pivot['event_prediction'] = effnet_stage_1_pivot.apply(lambda x : 'empty' if x['animal'] == 0 else 'animal', axis = 1)

In [100]:
effnet_stage_1_pivot = effnet_stage_1_pivot.reset_index()

In [101]:
empty_correct_count = effnet_stage_1_pivot[(~(effnet_stage_1_pivot['event_id'].str.contains('SSWI'))& 
                       (effnet_stage_1_pivot['event_prediction'] == 'empty'))].shape[0]

In [102]:
animal_correct_count = effnet_stage_1_pivot[((effnet_stage_1_pivot['event_id'].str.contains('SSWI'))& 
                       (effnet_stage_1_pivot['event_prediction'] == 'animal'))].shape[0]

In [103]:
(empty_correct_count + animal_correct_count)/ effnet_stage_1_pivot.shape[0]

0.92336217552534

# Phase 2- Species

## Yolo

In [7]:
#Read lines from txt results file

toppath = 'phase 2-species/'
txt_file = 'output_no_wolf.txt'

with open(toppath + txt_file, 'r') as f:
    original_lines = f.readlines()

In [8]:
#Txt file to dataframe
#We are taking the TOP confidence score species for each image as the species
#Count is the count for all bounding boxes drawn in image

rows = []

for item in original_lines:
    
    #Class number
    item_split_1 = item.split(' Bbox[list]')[0]
    class_conf_list = item_split_1.split(';')[1:-2]
    
    conf_score_max = 0
    
    i = 0    
    for class_conf in class_conf_list:
        conf_score_int = float(class_conf.split(',Conf:')[1])
        
        
        if conf_score_int > conf_score_max:
            conf_score_max = conf_score_int
            max_index = i
            
        i += 1
    try:
        class_string = class_conf_list[max_index].split(',')[0]
        class_num = float(class_string.split(':')[1])
        class_num = int(class_num)
    except:
        class_num = 'Blank'

    #Counts
    count_split_1 = item.split(' Bbox[list]')[-1]
    count_split_2 = count_split_1.replace(';', ',')  
    count_split_3 = count_split_2.split(',')
    count = int((len(count_split_3)-1)/4)
        
    #Filename
    file_split_1 = item.split('Filename: ')[1]
    file_name = file_split_1.split(';')[0].split('.jpg')[0]
        
    row = [file_name, class_num, count, conf_score_max]
    
    rows.append(row)       

predictions = pd.DataFrame(rows, columns = ['filename', 'class', 'count', 'conf_score_max'])

Species
1. Events with multiple images of same class will be labeled the majority class
- For event to be labeled Blank, all images must be blank
2. Events with all different labels will get labelled with highest confidence score 

Counts
1. Events with multiple images of same class- we will take the max of the majority class
2. Events with all different labels will get labelled with the count of highest conf score label


In [10]:
#Event column- remove A,B,C appendix
predictions['event_id'] = predictions['filename'].str[:-1]


## Events with muliple images of same class
#Species determination
#Groupby event, class
event_class_group_count = pd.DataFrame(predictions.groupby(['event_id', 'class']).count()['filename']).reset_index().rename(columns = {'filename': 'count_rows'})

#Filter by rows that have count greater than 1
event_class_group_count_majority = event_class_group_count[event_class_group_count['count_rows'] > 1]

#Remove blank events that are not a consensus for all 3 images
event_class_group_count_majority = event_class_group_count_majority[~((event_class_group_count_majority['class'] == 'Blank') &
                                (event_class_group_count_majority['count_rows'] < 3))]

#Count determination
event_class_group_majority_counts = pd.merge(predictions, event_class_group_count_majority,
         how = 'inner',
         on = ['event_id', 'class'])

event_class_majority_counts = event_class_group_majority_counts[['event_id', 'count']].groupby(['event_id']).max().reset_index()

event_max_conf_score = event_class_group_majority_counts[['event_id', 'conf_score_max']].groupby(['event_id']).max().reset_index()

majority_df = pd.merge(event_class_majority_counts, event_class_group_count_majority,
         on = 'event_id',
         how = 'left')[['event_id', 'count','class']]

majority_df = pd.merge(majority_df, event_max_conf_score,
         on = 'event_id',
         how = 'left')

## Events with all different labels
predictions_single_count = predictions[~predictions['event_id'].isin(majority_df['event_id'])]

event_list = []
pred_list = []
count_list = []
conf_score_max_list = []

previous_event = ''
conf_score_group_max = 0
pred_class = ''
conf_score_group_max = 0

predictions_single_count_grouped = predictions_single_count.groupby(['event_id', 'class',
                                                                     'conf_score_max'])

for group_name, group in predictions_single_count_grouped:
        
    current_event = group_name[0]
    current_class = group['class'].iloc[0]
    current_conf_score = group['conf_score_max'].iloc[0]
    current_count = group['count'].iloc[0]
    
    #Check if we are looking at a new event
    if current_event != previous_event:
        conf_score_max_list.append(conf_score_group_max)
        conf_score_group_max = 0
        event_list.append(previous_event)
        pred_list.append(pred_class)
        count_list.append(current_count)
        
    if conf_score_group_max < current_conf_score:
        pred_class = current_class
        conf_score_group_max = current_conf_score
        
    previous_event = current_event 

conf_score_max_list.append(conf_score_group_max)
event_list.append(previous_event)
pred_list.append(pred_class)
count_list.append(current_count)
    
single_class_df = pd.DataFrame(list(zip(event_list,
              pred_list,
              count_list,
              conf_score_max_list)),
            columns = ['event_id', 'class', 'count','conf_score_max'])

predictions_by_events_df = pd.concat([majority_df, single_class_df])


yolo_predictions_by_events_df = pd.merge(predictions_by_events_df, labels,
         how = 'left',
         on = 'class')

truth_pred_yolo_df = pd.merge(truth_file_no_other, yolo_predictions_by_events_df,
                         how = 'left',
                         left_on = 'event_id',
                         right_on = 'event_id')
yolo_preds_df = truth_pred_yolo_df[['event_id', 'class', 'conf_score_max', 'count', 'species']]

yolo_preds_df = yolo_preds_df.rename(columns = {'class': 'pred_class',
                                               'conf_score_max': 'conf_score',
                                                'count': 'pred_count',
                                               'species': 'pred_class_name'})




## EffNet, Megadetecter

In [11]:
model_results = pd.read_json('phase 2-species/phase2_efficientnetb5_classifications.json')
df = model_results['phase2_classification_results'].apply(pd.Series)
df['event_id'] = df['id'].str.split('.jpg').str[0]
df['event_id'] = df['event_id'].str[:-1]

In [12]:
## for index, row in df.iloc[:5].iterrows():
preds_dict = {}
for index, row in df.iterrows():
  event_id = str(row['event_id'])
  pred_class = row['class']
  pred_conf = row['conf']
  #print(event_id, pred_class, pred_conf)
  
  result_dict = {
        "class": pred_class,
        "conf": pred_conf
    }
  
  if event_id in preds_dict:
    preds_dict[event_id].append(result_dict)
  else:
    preds_dict[event_id] = [result_dict]

In [13]:

final_preds_dict = {}
for key, value in preds_dict.items():
  event_id = key
  counts = Counter(d['class'] for d in value)

  ## if all 3 predictions are different, defer to class with highest confidence
  if len(counts) == 3:
    highest_conf = max([x['conf'] for x in value])
    pred_class = [x['class'] for x in value if x['conf']==highest_conf][0]
  
  ## if there is an even number of predictions (2), defer to class with higher confidence
  elif sum(counts.values()) < 3:
    highest_conf = max([x['conf'] for x in value])
    pred_class = [x['class'] for x in value if x['conf']==highest_conf][0]

  ## otherwise, class is based on majority class, conf score is based on highest score for
  ## majority class
  else:
    most_common = {'most_common': counts.most_common(1)[0][0]}
    pred_class = most_common['most_common']
    highest_conf = max([x['conf'] for x in value if x['class'] == most_common['most_common']])

  final_preds_dict[event_id] = [pred_class, highest_conf]

In [14]:
eff_net_preds_df = pd.DataFrame.from_dict(final_preds_dict,  orient='index').reset_index()
eff_net_preds_df.columns=['event_id', 'pred_class', 'conf_score']
label_mapping = dict({0:'bear', 1:'cottontail_snowshoehare', 2:'coyote', 3:'deer', 4:'elk', 5:'foxgray_foxred', 6:'opossum', 7:'raccoon', 8:'turkey', 9:'wolf'})
eff_net_preds_df['pred_class_name'] = eff_net_preds_df['pred_class'].map(label_mapping)


## Consolidate Predictions

In [15]:
eff_net_preds_df['model'] = 'eff_net'
yolo_preds_df['model'] = 'yolo'

In [16]:
ensemble_pred_df = pd.concat([eff_net_preds_df, yolo_preds_df])

In [17]:
ensemble_max_conf_lookup = ensemble_pred_df[['event_id', 'conf_score']].groupby(['event_id']).max().reset_index()


In [18]:
ensemble_max_conf_pred_df =pd.merge(ensemble_pred_df,ensemble_max_conf_lookup,
         how = 'inner',
         on = ['event_id', 'conf_score'])

In [19]:
ensemble_result_metrics = pd.merge(truth_file_no_other, ensemble_max_conf_pred_df,
         how = 'left',
         on = ['event_id'])

In [20]:
y_true = ensemble_result_metrics['true_class_name']
y_pred = ensemble_result_metrics['pred_class_name']
label_list = np.unique(y_true)

target_names = label_list
print(classification_report(y_true, y_pred, target_names=target_names))


                         precision    recall  f1-score   support

                   bear       0.68      0.81      0.74       130
cottontail_snowshoehare       0.62      0.37      0.46       121
                 coyote       0.47      0.33      0.39       135
                   deer       0.82      0.34      0.48       191
                    elk       0.86      0.48      0.61       123
         foxgray_foxred       0.36      0.39      0.37        70
                opossum       0.83      0.38      0.53        52
                raccoon       0.67      0.41      0.51       119
                 turkey       0.47      0.69      0.56        64
                   wolf       0.28      0.87      0.42       126

               accuracy                           0.50      1131
              macro avg       0.61      0.51      0.51      1131
           weighted avg       0.62      0.50      0.51      1131



In [21]:
for model in ['eff_net', 'yolo']:
    ensemble_result_metrics_model = ensemble_result_metrics[ensemble_result_metrics['model'] == model]
    
    y_true = ensemble_result_metrics_model['true_class_name']
    y_pred = ensemble_result_metrics_model['pred_class_name']
    label_list = np.unique(y_true)

    target_names = label_list
    print(classification_report(y_true, y_pred, target_names=target_names))

                         precision    recall  f1-score   support

                   bear       0.78      0.82      0.80       109
cottontail_snowshoehare       0.42      0.23      0.30        83
                 coyote       0.29      0.11      0.16        84
                   deer       0.82      0.21      0.33       136
                    elk       0.94      0.46      0.62        96
         foxgray_foxred       0.22      0.21      0.21        43
                opossum       0.76      0.33      0.46        39
                raccoon       0.56      0.27      0.37        73
                 turkey       0.49      0.67      0.56        55
                   wolf       0.28      0.95      0.43       116

               accuracy                           0.45       834
              macro avg       0.56      0.43      0.42       834
           weighted avg       0.58      0.45      0.44       834

                         precision    recall  f1-score   support

                   be

/Users/sleung2/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sleung2/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sleung2/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:

yolo_metrics = pd.merge(yolo_preds_df, truth_file_no_other, 
         how = 'left',
         on = ['event_id'])


In [23]:
yolo_metrics_not_null = yolo_metrics[~yolo_metrics['pred_class_name'].isnull()]
yolo_metrics_not_null = yolo_metrics_not_null[yolo_metrics_not_null['true_class_name'] != 'wolf']


In [24]:

y_true = yolo_metrics_not_null['true_class_name']
y_pred = yolo_metrics_not_null['pred_class_name']
label_list = np.unique(y_true)

target_names = label_list
print(classification_report(y_true, y_pred, target_names=target_names))

    
    

                         precision    recall  f1-score   support

                   bear       0.63      0.92      0.75       127
cottontail_snowshoehare       0.87      0.65      0.75       106
                 coyote       0.56      0.65      0.60       131
                   deer       0.75      0.56      0.64       176
                    elk       0.76      0.50      0.61       115
         foxgray_foxred       0.35      0.51      0.42        69
                opossum       0.82      0.69      0.75        45
                raccoon       0.73      0.66      0.69       106
                 turkey       0.64      0.83      0.72        60

               accuracy                           0.66       935
              macro avg       0.68      0.66      0.66       935
           weighted avg       0.69      0.66      0.66       935

